In [1]:
import os
import pandas as pd
import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.DataFrame()
for f in [x for x in os.listdir("data") if x[2] == "_"]:
    df = pd.concat([df, pd.read_parquet(os.path.join("data",f))])

In [3]:
df['oscrapovano_minuty'] = df['oscrapovano'].apply(lambda x: str(x)[0:14])

In [4]:
df['oscrapovano_minuty']

0      2024-11-07 18:
1      2024-11-07 18:
2      2024-11-07 18:
3      2024-11-07 18:
4      2024-11-07 18:
            ...      
457    2025-04-02 19:
458    2025-04-02 19:
459    2025-04-02 07:
460    2025-04-02 07:
461    2025-04-02 07:
Name: oscrapovano_minuty, Length: 7234264, dtype: object

In [5]:
df = df.drop_duplicates(subset=['odkud','kam','odjezd','oscrapovano_minuty'], keep="last")

In [6]:
len(df)

2984143

In [7]:
df[['odjezd','odkud','kam']].drop_duplicates().shape[0]

189073

In [57]:
df.groupby("prodejce")['oscrapovano'].max()

C:\Users\micha\AppData\Local\Temp\ipykernel_18096\1446830039.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("prodejce")['oscrapovano'].max()


prodejce
ARR   2025-04-02 22:32:20
LE    2025-04-02 22:24:02
RJ    2025-04-02 22:12:28
ČD    2025-04-02 22:08:35
Name: oscrapovano, dtype: datetime64[ns]

In [8]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-07      2297
          2024-11-08      7128
          2024-11-09      3320
          2024-11-10      3157
          2024-11-11      3367
                         ...  
ČD        2025-03-29     12359
          2025-03-30     12022
          2025-03-31     12577
          2025-04-01     12048
          2025-04-02     11415
Length: 588, dtype: int64

Výpadek ČD 2024-11-26 opraven, vyřešen.

In [10]:
df = df.sort_values(by="oscrapovano").reset_index(drop=True)

In [11]:
df = df.dropna(subset=['odkud','kam','odjezd','oscrapovano'],how='any')

In [12]:
df = df[df['prostredek'] != 'autobus']

In [13]:
df = df[~df['kam'].str.contains('(tief)',na=False)]
df = df[~df['odkud'].str.contains('(tief)',na=False)]

C:\Users\micha\AppData\Local\Temp\ipykernel_18096\103445691.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['kam'].str.contains('(tief)',na=False)]
C:\Users\micha\AppData\Local\Temp\ipykernel_18096\103445691.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['odkud'].str.contains('(tief)',na=False)]


Před 7. listopadem scrapování testovací a nesystematické = nevypovídající = filtrujeme.

In [15]:
df = df[df['oscrapovano'] >= '2024-11-08']

In [16]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-08      7128
          2024-11-09      3320
          2024-11-10      3157
          2024-11-11      3367
          2024-11-12      3323
                         ...  
ČD        2025-03-29     12176
          2025-03-30     12007
          2025-03-31     12218
          2025-04-01     11595
          2025-04-02     11214
Length: 576, dtype: int64

In [17]:
days = {0: 'po', 1: 'út', 2: 'st', 3: 'čt', 
        4: 'pá', 5: 'so', 6: 'ne'}
df['den'] = df['odjezd'].dt.dayofweek.map(days)

In [18]:
df['predstih_d'] = df['predstih'].dt.days
df['predstih_h'] = df['predstih'].dt.total_seconds() / 3600

In [19]:
df = df[df['predstih_h'] > -3]

In [20]:
kategoricka_data = ['odkud','kam','prodejce','den']

In [21]:
for k in kategoricka_data:
    print("Before:", df[k].memory_usage(deep=True))
    df[k] = df[k].astype('category')
    print("After: ", df[k].memory_usage(deep=True))

Before: 184354902
After:  23462042
Before: 184378606
After:  23457512
Before: 198016054
After:  23448991
Before: 172644173
After:  23449311


In [22]:
df['cena'] = pd.to_numeric(df['cena'])

In [23]:
df.shape[0]

2605399

In [24]:
df[['odjezd','odkud','kam']].drop_duplicates().shape[0]

167920

In [25]:
poradi = ['oscrapovano','prodejce','odkud','kam',
 'odjezd',
 'predstih',
 'predstih_d',
 'predstih_h',
          'cena',
 'prostredek',
          'volnych_mist',
 'obsazenost',
 'jizdni_doba',
 'vzdalenost',
 'zpozdeni',
 'cena_poznamka',
 'den',
 'prestupy',
'vlaky',
 'mistenka_zdarma',
 'nahradni_bus',
 'volna_mista_economy',
 'volna_mista_economy_plus',
 'volna_mista_economy_business',
 'volna_mista_premium']

In [26]:
df[poradi].to_parquet(os.path.join("data","jizdenky.parquet"))

In [27]:
nejnovejsi = df['oscrapovano'].max()
nejnovejsi
df_tyden = df[df['oscrapovano'] > (nejnovejsi - datetime.timedelta(hours=168))]

In [28]:
df_tyden[poradi].to_csv(os.path.join("data","jizdenky_tyden.csv"))